In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

def load_frame_metrics(filename="frame_metrics.pkl"):
    """
    Load the nested dictionary containing multiple metrics per video from a pickle file.
    Structure: {cube -> {condition -> {line -> {metric -> 1D array}}}}
    """
    with open(filename, "rb") as f:
        data = pickle.load(f)
    print(f"Loaded frame metrics from {filename}")
    return data

def plot_metric(results, cube, condition, line, metric_name, window_size=50):
    """
    Plot evolution of a chosen metric showing both raw and filtered signals.
    The raw signal is colored based on thresholds calculated from filtered data.
    
    Parameters
    ----------
    results : dict
        Pre-computed results dictionary
    cube : int
        Cube number
    condition : int
        Condition number
    line : int
        Line number
    metric_name : str
        Name of the metric to plot ('intensities', 'saturated_pixels', or 'max_values')
    window_size : int
        Size of the moving average window for filtering
    """
    signal = results[cube][condition][line][metric_name]
    frames = range(len(signal))
    
    # Filter signal for threshold calculation (centered moving average)
    filtered_signal = np.convolve(signal, np.ones(window_size)/window_size, mode='same')
    
    # Calculate threshold using filtered signal
    p90 = np.percentile(filtered_signal, 99)
    p10 = np.percentile(filtered_signal, 1)
    th = (p90-p10) * 0.3 + p10
    
    print(f"99th percentile: {p90:.2f}")
    print(f"1st percentile: {p10:.2f}")
    print(f"Calculated threshold: {th:.2f}")
    
    ylabel_map = {
        'intensities': 'Sum of pixel intensities',
        'saturated_pixels': 'Number of saturated pixels',
        'max_values': 'Maximum pixel value'
    }
    
    plt.figure(figsize=(6,4))
    
    # Plot raw signal values based on filtered threshold
    mask_below = [x <= th for x in signal]
    plt.plot(frames, [s if m else None for s, m in zip(signal, mask_below)], 
             color='blue', label='Normal')
    
    mask_above = [x > th for x in signal]
    plt.plot(frames, [s if m else None for s, m in zip(signal, mask_above)], 
             color='red', label='Above threshold')
    
    # Plot filtered signal
    plt.plot(frames, filtered_signal, color='green', alpha=0.5, 
             linestyle='--', label='Filtered signal')
    
    plt.axhline(y=th, color='gray', linestyle='--', 
                label=f'Threshold (filtered signal)')
    plt.xlabel('Frame index')
    plt.ylabel(ylabel_map[metric_name])
    plt.title(f'Evolution of {metric_name.replace("_", " ")} over time')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_all_metrics(results, cube, condition, line):
    """
    Create a figure with subplots showing all metrics for a specific video.
    
    Parameters
    ----------
    results : dict
        The nested dictionary containing the metrics
    cube : int
        Cube number
    condition : int
        Condition number
    line : int
        Line number
    """
    metrics = ['intensities', 'saturated_pixels', 'max_values']
    
    fig, axes = plt.subplots(3, 1, figsize=(8, 10))
    fig.suptitle(f'Metrics Evolution: Cube={cube}, Condition={condition}, Line={line}')
    
    ylabel_map = {
        'intensities': 'Sum of pixel intensities',
        'saturated_pixels': 'Number of saturated pixels',
        'max_values': 'Maximum pixel value'
    }
    
    for ax, metric in zip(axes, metrics):
        signal = results[cube][condition][line][metric]
        ax.plot(signal)
        ax.set_xlabel('Frame index')
        ax.set_ylabel(ylabel_map[metric])
        ax.grid(True)
    
    plt.tight_layout()
    plt.show()

def plot_metric_comparison(results, cube, condition, metrics_to_compare=None):
    """
    Plot the same metric for all lines in a given condition, allowing for comparison.
    
    Parameters
    ----------
    results : dict
        The nested dictionary containing the metrics
    cube : int
        Cube number
    condition : int
        Condition number
    metrics_to_compare : list of str, optional
        List of metrics to compare. If None, plots all metrics.
    """
    if metrics_to_compare is None:
        metrics_to_compare = ['intensities', 'saturated_pixels', 'max_values']
    
    ylabel_map = {
        'intensities': 'Sum of pixel intensities',
        'saturated_pixels': 'Number of saturated pixels',
        'max_values': 'Maximum pixel value'
    }
    
    n_metrics = len(metrics_to_compare)
    fig, axes = plt.subplots(n_metrics, 1, figsize=(10, 4*n_metrics))
    if n_metrics == 1:
        axes = [axes]
    
    fig.suptitle(f'Metrics Comparison: Cube={cube}, Condition={condition}')
    
    for ax, metric in zip(axes, metrics_to_compare):
        for line in results[cube][condition].keys():
            signal = results[cube][condition][line][metric]
            ax.plot(signal, label=f'Line {line}')
        
        ax.set_xlabel('Frame index')
        ax.set_ylabel(ylabel_map[metric])
        ax.legend()
        ax.grid(True)
    
    plt.tight_layout()
    plt.show()

In [2]:
results = load_frame_metrics("frame_metrics.pkl")


Loaded frame metrics from frame_metrics.pkl


In [ ]:
plot_metric(results, cube=1, condition=1, line=1, metric_name='max_values')


In [ ]:
plot_metric(results, cube=2, condition=2, line=2, metric_name='max_values')


In [ ]:
plot_metric(results, cube=3, condition=3, line=3, metric_name='max_values')


In [ ]:
plot_metric(results, cube=4, condition=4, line=4, metric_name='max_values')


In [ ]:
# Loop through all cubes, conditions, and lines
for cube in range(1, 5):         # Cubes: 1 to 4
    for condition in range(1, 13):  # Conditions: 1 to 12
        for line in range(1, 6):     # Lines: 1 to 5
            # Check if the data exists before plotting
            if (cube in results and 
                condition in results[cube] and 
                line in results[cube][condition]):
                
                print(f"Plotting: Cube={cube}, Condition={condition}, Line={line}")
                plot_metric(results, cube=cube, condition=condition, line=line, metric_name='max_values')
            else:
                print(f"Skipping: Cube={cube}, Condition={condition}, Line={line} (No Data Found)")


In [3]:
import os
import numpy as np
import tifffile
import pickle
import shutil
from pathlib import Path

def find_segmentation_range(signal, threshold_ratio=0.3, window_size=50):
    """
    Given a 1D signal, find the first and last index crossing a threshold
    calculated from the filtered signal data.
    """
    if len(signal) == 0:
        return None, None
        
    # Filter signal for threshold calculation (centered moving average)
    filtered_signal = np.convolve(signal, np.ones(window_size)/window_size, mode='same')
    
    # Calculate threshold using filtered signal
    p90 = np.percentile(filtered_signal, 99)
    p10 = np.percentile(filtered_signal, 1)
    th = (p90-p10) * threshold_ratio + p10
    
    # Find the first index crossing threshold
    start_idx_candidates = np.where(signal >= th)[0]
    if len(start_idx_candidates) == 0:
        return None, None
        
    return start_idx_candidates[0], start_idx_candidates[-1]

def get_video_segment(
    frame_metrics,
    cube,
    condition,
    line,
    data_dir="raw_data",
    threshold_ratio=0.3,
    metric_name='max_values',
    window_size=50
):
    """
    Load a single segmented video for specific cube/condition/line.
    """
    try:
        signal = frame_metrics[cube][condition][line][metric_name]
    except KeyError:
        print(f"Data not found for cube={cube}, condition={condition}, line={line}")
        return None, None, None
        
    # Find segmentation boundaries
    start_idx, end_idx = find_segmentation_range(
        signal, 
        threshold_ratio=threshold_ratio,
        window_size=window_size
    )
    
    if start_idx is None or end_idx is None:
        print(f"No frames above threshold for metric '{metric_name}'")
        return None, None, None

    # Construct the TIF path
    file_path = os.path.join(
        data_dir, 
        f"cube{cube}", 
        f"cube{cube}_C001H001S00{condition:02d}",
        f"cube{cube}_C001H001S00{condition:02d}-{(line-1):02d}.tif"
    )

    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        return None, None, None

    print(f"Loading frames {start_idx} to {end_idx} using metric '{metric_name}'")

    # Read frames one at a time to reduce memory usage
    frames_list = []
    with tifffile.TiffFile(file_path) as tif:
        total_frames = len(tif.pages)
        if end_idx >= total_frames:
            end_idx = total_frames - 1
        
        for i in range(start_idx, end_idx + 1):
            frame = tif.pages[i].asarray().astype(float)
            frames_list.append(frame)

    video_array = np.stack(frames_list, axis=0)
    return video_array, start_idx, end_idx

def iter_video_segments(
    frame_metrics,
    data_dir="raw_data",
    threshold_ratio=0.3,
    metric_name='max_values',
    window_size=50
):
    """
    Generator that yields one segmented video at a time.
    """
    for cube in frame_metrics:
        for condition in frame_metrics[cube]:
            for line in frame_metrics[cube][condition]:
                result = get_video_segment(
                    frame_metrics,
                    cube,
                    condition,
                    line,
                    data_dir=data_dir,
                    threshold_ratio=threshold_ratio,
                    metric_name=metric_name,
                    window_size=window_size
                )
                
                if result[0] is not None:
                    yield cube, condition, line, *result

def process_videos(frame_metrics, data_dir="raw_data", output_dir="segmented_videos_data"):
    """
    Process and save videos one at a time to minimize memory usage.
    """
    # Create output directory structure
    base_dir = Path(output_dir)
    if base_dir.exists():
        shutil.rmtree(base_dir)
    base_dir.mkdir(parents=True, exist_ok=True)
    
    # Initialize metadata dictionary
    metadata = {}
    
    # Count total videos to process
    total_videos = sum(
        1 for cube in frame_metrics 
        for condition in frame_metrics[cube] 
        for line in frame_metrics[cube][condition]
    )
    processed = 0
    
    try:
        # Process one video at a time
        for cube, condition, line, video_array, start_idx, end_idx in iter_video_segments(
            frame_metrics,
            data_dir=data_dir
        ):
            try:
                # Create directory structure for this video
                cube_dir = base_dir / f"cube_{cube}"
                cube_dir.mkdir(exist_ok=True)
                
                cond_dir = cube_dir / f"condition_{condition}"
                cond_dir.mkdir(exist_ok=True)
                
                # Initialize metadata structure if needed
                if cube not in metadata:
                    metadata[cube] = {}
                if condition not in metadata[cube]:
                    metadata[cube][condition] = {}
                
                # Save individual video file
                video_filename = cond_dir / f"line_{line}.pkl"
                with open(video_filename, "wb") as f:
                    pickle.dump(video_array, f)
                
                # Store file location in metadata
                metadata[cube][condition][line] = str(video_filename.relative_to(base_dir))
                
                # Free memory
                del video_array
                import gc
                gc.collect()
                
                processed += 1
                print(f"Processed: Cube {cube}, Condition {condition}, Line {line} [{processed}/{total_videos}]")
                
            except Exception as e:
                print(f"Error processing video (Cube {cube}, Condition {condition}, Line {line}): {str(e)}")
                continue
        
        # Save metadata file
        metadata_file = base_dir / "metadata.pkl"
        with open(metadata_file, "wb") as f:
            pickle.dump(metadata, f)
    
            
    except Exception as e:
        print(f"Error during processing: {str(e)}")
        if base_dir.exists():
            shutil.rmtree(base_dir)
        raise

In [4]:
# Process the videos
process_videos(
    results,
    data_dir="raw_data",
    output_dir="segmented_videos_data"
)

Loading frames 755 to 1029 using metric 'max_values'
Processed: Cube 1, Condition 1, Line 1 [1/240]
Loading frames 1073 to 1347 using metric 'max_values'
Processed: Cube 1, Condition 1, Line 2 [2/240]
Loading frames 1390 to 1664 using metric 'max_values'
Processed: Cube 1, Condition 1, Line 3 [3/240]
Loading frames 1708 to 1981 using metric 'max_values'
Processed: Cube 1, Condition 1, Line 4 [4/240]
Loading frames 2025 to 2299 using metric 'max_values'
Processed: Cube 1, Condition 1, Line 5 [5/240]
Loading frames 755 to 970 using metric 'max_values'
Processed: Cube 1, Condition 2, Line 1 [6/240]
Loading frames 956 to 1172 using metric 'max_values'
Processed: Cube 1, Condition 2, Line 2 [7/240]
Loading frames 1158 to 1373 using metric 'max_values'
Processed: Cube 1, Condition 2, Line 3 [8/240]
Loading frames 1359 to 1575 using metric 'max_values'
Processed: Cube 1, Condition 2, Line 4 [9/240]
Loading frames 1561 to 1776 using metric 'max_values'
Processed: Cube 1, Condition 2, Line 5 [1

Processed: Cube 2, Condition 5, Line 2 [82/240]
Loading frames 1158 to 1374 using metric 'max_values'
Processed: Cube 2, Condition 5, Line 3 [83/240]
Loading frames 1360 to 1575 using metric 'max_values'
Processed: Cube 2, Condition 5, Line 4 [84/240]
Loading frames 1561 to 1777 using metric 'max_values'
Processed: Cube 2, Condition 5, Line 5 [85/240]
Loading frames 755 to 1029 using metric 'max_values'
Processed: Cube 2, Condition 6, Line 1 [86/240]
Loading frames 1074 to 1346 using metric 'max_values'
Processed: Cube 2, Condition 6, Line 2 [87/240]
Loading frames 1390 to 1663 using metric 'max_values'
Processed: Cube 2, Condition 6, Line 3 [88/240]
Loading frames 1708 to 1981 using metric 'max_values'
Processed: Cube 2, Condition 6, Line 4 [89/240]
Loading frames 2025 to 2299 using metric 'max_values'
Processed: Cube 2, Condition 6, Line 5 [90/240]
Loading frames 762 to 1392 using metric 'max_values'
Processed: Cube 2, Condition 7, Line 1 [91/240]
Loading frames 1748 to 2368 using me

Processed: Cube 3, Condition 9, Line 2 [162/240]
Loading frames 1390 to 1664 using metric 'max_values'
Processed: Cube 3, Condition 9, Line 3 [163/240]
Loading frames 1708 to 1989 using metric 'max_values'
Processed: Cube 3, Condition 9, Line 4 [164/240]
Loading frames 2025 to 2298 using metric 'max_values'
Processed: Cube 3, Condition 9, Line 5 [165/240]
Loading frames 757 to 1359 using metric 'max_values'
Processed: Cube 3, Condition 10, Line 1 [166/240]
Loading frames 1735 to 2336 using metric 'max_values'
Processed: Cube 3, Condition 10, Line 2 [167/240]
Loading frames 2712 to 3255 using metric 'max_values'
Processed: Cube 3, Condition 10, Line 3 [168/240]
Loading frames 0 to 59 using metric 'max_values'
Processed: Cube 3, Condition 10, Line 4 [169/240]
Loading frames 434 to 1056 using metric 'max_values'
Processed: Cube 3, Condition 10, Line 5 [170/240]
Loading frames 755 to 1132 using metric 'max_values'
Processed: Cube 3, Condition 11, Line 1 [171/240]
Loading frames 1279 to 165